# Trabajo Práctico 1: Enunciado


El trabajo práctico 1 de la materia se basa en el análisis de los tweets del set de datos de la competencia: https://www.kaggle.com/c/nlp-getting-started.  

- id - identificador unico para cada  tweet
- text - el texto del tweet
- location - ubicación desde donde fue enviado (podría no estar)
- keyword - un keyword para el tweet  (podría faltar)
- target - en train.csv, indica si se trata de un desastre real  (1) o no (0)

El objetivo del primer TP es realizar un análisis exploratorio del set de datos. Queremos ver qué cosas podemos descubrir sobre los datos que puedan resultar interesantes. Estas cosas pueden estar relacionadas al objetivo del TP2 (predecir si un cierto tweet es real o no) o no, ambas son de interés.

Los requisitos de la primera entrega son los siguientes:

- El análisis debe estar hecho en Python Pandas o R.
- El análisis debe entregarse en formato pdf vía gradescope. En el informe no va código.
- Informar el link a un repositorio Github en donde pueda bajarse el código completo para generar el análisis.

La evaluación del TP se realizará en base al siguiente criterio:

- Originalidad del análisis exploratorio. 
- Calidad del reporte. ¿Está bien escrito? ¿Es claro y preciso? 
- Calidad del análisis exploratorio: qué tipo de preguntas se hacen y de qué forma se responden, ¿es la respuesta clara y concisa con respecto a la pregunta formulada? 
- Calidad de las visualizaciones presentadas.
  - ¿Tienen todos los ejes su rótulo?
  - ¿Tiene cada visualización un título?
  - ¿Es entendible la visualización sin tener que leer la explicación?
  - ¿El tipo de plot elegido es adecuado para lo que se quiere visualizar?
  - ¿Es una visualización interesante?
  - ¿El uso del color es adecuado?
  - ¿Hay un exceso o falta de elementos visuales en la visualización elegida?
  - ¿La visualización es consistente con los datos?
- Conclusiones presentadas.


In [ ]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
tweets = pd.read_csv('../data/train.csv', encoding='utf-8')
tweets.info()

In [ ]:
# IMPORTANTE esto reemplazando todos los NaN por None
tweets = tweets.fillna('None')

### Normalización de la columna text y creación de nuevas columnas


In [ ]:
# primera normalización del texto, pasamos a lowercase 
tweets['normalized_text'] = tweets.text.str.lower()

#### Eliminamos los links del texto y dejandolos en una columna a parte por si sirven a futuro

In [ ]:
# nueva columna con el total de links
import re
URLPATTERN = r'(https?://\S+)' 

tweets['urls'] = tweets.normalized_text.apply(lambda x: re.findall(URLPATTERN, x))
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(URLPATTERN,"", x))
# cuento la cantidad de links en los tweets
tweets['url_count'] = tweets.urls.str.len()

In [ ]:
# vemos como queda el dataset
tweets[(tweets['url_count'] > 1)].head()

#### Eliminamos los hashtags del texto y lo dejamos en otra columna junto con el count

In [ ]:
# cuento la cantidad de hashtags en los tweets
# nueva columna con el total de hashtags, y los hashtags
tweets['hashtags'] = tweets.normalized_text.apply(lambda x: re.findall(r"#(\w+)", x))
tweets['hashtags_count'] = tweets.hashtags.str.len()
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(r"#(\w+)","", x))

In [ ]:
# vemos como queda el dataset
tweets[(tweets['hashtags_count'] > 1)].head()

#### Eliminamos los tags del texto y lo dejamos en otra columna junto con el count

In [ ]:
# cuento la cantidad de ags en los tweets
# nueva columna con el total de tags, y los tags
tweets['tags'] = tweets.text.str.lower().apply(lambda x: re.findall(r"@(\w+)", x))
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(r"@(\w+)","", x))
tweets['tags_count'] = tweets.tags.str.len()

In [ ]:
# vemos como queda el dataset
tweets[(tweets['tags_count'] > 1)].head()

#### Eliminamos signos de puntuación y html

In [ ]:
import string
translator = str.maketrans('', '', string.punctuation)

# !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~

def remove_punctuation(text):        
    return text.translate(translator)

In [ ]:
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup
from html import unescape

In [ ]:
def remove_htmlsymbols(text):
    soup = BeautifulSoup(unescape(text))
    return soup.text

In [ ]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

def remove_emojis_non_ascii(text):    
    #replace consecutive non-ASCII characters with a space
    result = re.sub(r'[^\x00-\x7F]+',' ', text)
    #remove emojis from tweet
    result = emoji_pattern.sub(r'', result)    
    return result

In [ ]:
tweets['normalized_text'] = tweets.normalized_text.apply(remove_htmlsymbols)
tweets['normalized_text'] = tweets.normalized_text.apply(remove_punctuation)
tweets['normalized_text'] = tweets.normalized_text.apply(remove_emojis_non_ascii)

#### Eliminamos stop words y creamos nueva columna con array de palabras

In [ ]:
!pip install nltk
!pip install stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

In [ ]:
# convertimos el texto en listado de palabras y despues borramos las stop words
tweets['words'] = tweets.normalized_text.str.split()
stop_words = stopwords.words('english')
tweets['normalized_words'] = tweets['words'].apply(lambda x: [item for item in x if item not in stop_words])
# eliminar las stop words del texto normalizado
tweets['normalized_text'] = [' '.join(map(str, l)) for l in tweets['normalized_words']]
# vemos como queda el dataset
tweets.head()

#### Analizamos si el texto tiene números y guardamos el dato en una nueva columna

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# esto chequea números como 1,2 y tambíen escrito como one, two
def existence_of_numeric_data(text):
    text=nltk.word_tokenize(text)
    pos = nltk.pos_tag(text)
    count = 0
    for i in range(len(pos)):
        word , pos_tag = pos[i]
        if pos_tag == 'CD':
            return 1
    return 0


In [ ]:
# con 1 si tiene , 0 sino
tweets['has_numbers'] = tweets.normalized_text.apply(existence_of_numeric_data)
# vemos como queda el dataset
tweets[(tweets['has_numbers'] == 1)].head()

### Normalización de la columna keyword

In [ ]:
# eliminamos "%20" que representa espacio
tweets['keywords'] = tweets.keyword.str.replace('%20',' ')

## Visualizaciones

#### Relación entre cantidad de links y target

In [ ]:
sns.catplot(x="target", y="url_count",data=tweets)

In [ ]:
sns.catplot(x="target", y="url_count", kind="box", data=tweets);

#### Relación entre tweet con números y target

In [ ]:
tweets['target_name']= np.where(tweets['target']==1 ,'SI','NO')
tweets['has_numbers_name']= np.where(tweets['has_numbers']==1 ,'SI','NO')

In [ ]:
plot_tweets_numbers = sns.countplot(data=tweets,x="target_name", hue="has_numbers_name", palette="cubehelix")
plot_tweets_numbers.set_title("Cantidad de tweets que contienen números y su veracidad", fontsize=18)
plot_tweets_numbers.set_xlabel("Real?", fontsize=14)
plot_tweets_numbers.set_ylabel("Cantidad de tweets", fontsize=14)
plot_tweets_numbers.legend(loc='upper left', title='Tweets con números?')


### Wordcloud con texto normalizado

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator

Wordcloud para textos fake

In [ ]:
# me quedo con todos los textos fakes
fakes= tweets[tweets.target == 0]
fakes_text = " ".join(review for review in fakes.normalized_text)

In [ ]:
# Create and generate a word cloud image:
wordcloud_fake = WordCloud(max_font_size=80, max_words=500, background_color="white").generate(fakes_text)

# Display the generated image:
plt.imshow(wordcloud_fake, interpolation='bilinear')
plt.axis("off")
plt.show()
wordcloud_fake.to_file("wordcloud_fake.png")


Wordcloud para textos reales

In [ ]:
# me quedo con todos los textos reales
reales= tweets[tweets.target == 1]
reales_text = " ".join(review for review in reales.normalized_text)

In [ ]:
#?WordCloud

In [ ]:
# Create and generate a word cloud image:
wordcloud_real = WordCloud(max_font_size=80, max_words=500, background_color="white").generate(reales_text)

# Display the generated image:
plt.imshow(wordcloud_real, interpolation='bilinear')
plt.axis("off")
plt.show()
wordcloud_real.to_file("wordcloud_real.png")


### Top words (WIP)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
# Collect vocabulary count
# create a count vectorizer object
count_vectorizer = CountVectorizer()
# fit the count vectorizer using the text data
count_vectorizer.fit(tweets.normalized_text)
# collect the vocabulary items used in the vectorizer
dictionary = count_vectorizer.vocabulary_.items()  

In [ ]:
# lists to store the vocab and counts
vocab = []
count = []
# iterate through each vocab and count append the value to designated lists
for key, value in dictionary:
    vocab.append(key)
    count.append(value)
# store the count in pandas dataframe with vocab as index
vocab_bef_stem = pd.Series(count, index=vocab)
# sort the dataframe
vocab_bef_stem = vocab_bef_stem.sort_values(ascending=False)

In [ ]:
vocab_bef_stem.head(20)

In [ ]:
top_vacab = vocab_bef_stem.head(50)
top_vacab.plot(kind = 'barh', figsize=(5,10))

#### Stemming operations


Stemming operation bundles together words of same root. E.g. stem operation bundles "response" and "respond" into a common "respon"

In [ ]:
from nltk.stem.snowball import SnowballStemmer

# create an object of stemming function
stemmer = SnowballStemmer("english")

def stemming(words):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in words]
    return " ".join(text) 

In [ ]:
tweets['stemmed_text'] = tweets['normalized_words'].apply(stemming)
tweets[['target','stemmed_text',  'normalized_text', 'text']].head(10)

Top words after stemming operation


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# create the object of tfid vectorizer
tfid_vectorizer = TfidfVectorizer("english")
# fit the vectorizer using the text data
tfid_vectorizer.fit(tweets['stemmed_text'])
# collect the vocabulary items used in the vectorizer
dictionary = tfid_vectorizer.vocabulary_.items()  

In [ ]:
vocab = []
count = []
# iterate through each vocab and count append the value to designated lists
for key, value in dictionary:
    vocab.append(key)
    count.append(value)
# store the count in panadas dataframe with vocab as index
vocab_after_stem = pd.Series(count, index=vocab)
# sort the dataframe
vocab_after_stem = vocab_after_stem.sort_values(ascending=False)
# plot of the top vocab
top_vacab = vocab_after_stem.head(20)
top_vacab.plot(kind = 'barh', figsize=(5,10))

#### Bag of Words

In [ ]:
from collections import Counter
# contamos la cantidad de palabras de cada tweet
tweets['words_counter'] = tweets.normalized_words.apply(Counter)

In [ ]:
tweets.head()

In [ ]:
# me quedo con las palabras que mas ocurrencias tienen en cada row
tweets['word_max_appearance'] = tweets.words_counter.apply( lambda x: max(x) if x else None)  
tweets.head()

In [ ]:
tweets['word_max_appearance_count'] =  tweets.words_counter.apply( lambda x: max(x.values()) if x else 0)
tweets.head()

In [ ]:
tweets['word_max_appearance_count'].value_counts()

In [ ]:
# me quedo con las palabras que mas ocurrencias tienen en todo el dataset para los fake

In [ ]:
# me quedo con las palabras que mas ocurrencias tienen en todo el dataset para los posta

In [ ]:
tweets.head()

### Análisis de los keywords

In [ ]:
tweets.keywords.value_counts()

In [ ]:
# Agrupo por keyword y cuento la cantidad de apariciones que tienen y el promedio del target para sacar una suerte de "probabilidad de catastrofe"
ag = tweets.groupby('keywords').agg({'text':'count', 'target':'mean'}).rename(columns={'text':'Apariciones', 'target':'Probabilidad de Catastrofe'})
ag.sort_values('Probabilidad de Catastrofe', ascending=False).head(20)


In [ ]:
probability = tweets.groupby('keywords').agg({'target':'mean'}).rename(columns={'target':'Probabilidad de Catastrofe'})
probability.sort_values('Probabilidad de Catastrofe', ascending=False).head(20)

In [ ]:
probability.sort_values('Probabilidad de Catastrofe', ascending=False).head(50).plot(kind='bar',figsize=(16,8),rot=85,title='Probabilidad de catástrofe según keyword')
ax=plt.gca()
ax.set_ylabel('Probabilidad');
ax.set_xlabel('Keyword')

In [ ]:
# me guardo el listado de keywords con mayor probabilidad de que resuleten en un tweet verdadero
probability_keywords_df = probability[probability['Probabilidad de Catastrofe'] >= 0.85].sort_values('Probabilidad de Catastrofe', ascending=False).reset_index()
common_keywords_real_disaster = probability_keywords_df.keywords
common_keywords_real_disaster